# PROJET DE PYTHON
# THEME : ANALYSE DE LA SATISFACTION DES UTILISATEURS DE LAPTOPS : CAS DES LAPTOPS DELL
# Auteurs:
* MECHIDE SERINE
* METHAFE KUITE SORELLE LOVELINE
* NKAMENI DANIEL
* POUANI EMAPI HORNELLA JOÊLLE

# SCRAPING DES DONNEES SUR LES LAPTOPS DELL

Le but de cette partie est de scraper tous les informations (caractéristiques et commentaires) sur les machines proposées sur le site officiel de Dell France au 10 Dcembre 2021

## SCRAPING DES COMMENTAIRES SUR LES LAPTOPS DELL

Pour scraper les commentaires de chaque machine, il incombe d'ouvrir individuellement les pages de chacune d'elles. Les liens vers la page détaillée de chaque machine sont obtenu à l'aide d'un premier craping. Les pages détaillées des machines sont dynamiques et chargées avec Java Script. Les méthodes de scraping basic vu en cours ne suffiront donc pas dans ce cas. Nous utilisons donc dans la suite le package Selenium adapt au scraping des pages dynamique.

Le pilote Web utilisé est Chrome. l'étape suivante est donc de configurer ce pilote. On définit également un temps d'attente necessaire au chargement de chaque page. Nous avons défini un temps d'attente de 5 secondes.

In [3]:
# Importation des packages

from bs4 import BeautifulSoup 
from selenium import webdriver
import requests  
import re
import time  
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains 
from joblib import Parallel, delayed 
import itertools
import urllib
import bs4
import pandas as pd
from urllib import request

In [ ]:
## Installer chrome driver
driver_path = 'C:/Users/nkame/anaconda3/chromedriver.exe'
opt = webdriver.ChromeOptions()
opt.add_experimental_option('w3c', False)
driver = webdriver.Chrome(executable_path=driver_path,options=opt)


##Initialiser le driver de Selenium
def setupDriver(url, waiting_time = 5):
	driver = webdriver.Chrome(options=opt)
	driver.get(url)
	time.sleep(waiting_time) 
	return driver

La page de chaque machine affiche les premiers commentaires de cette macine en bas de page. Pour les machines dontles commentaires ne tiennent pas sur une page, il faut retrouver l'url de la page suivante et l'utiliser pour acceder aux commentaires suivants. 

Attention: Pour la première page de commentaires comme les page suivante, il est important d'entrer exactement l'url de la section commentaire sinon l'extraction n'aura pas lieu.

Le code suivant permet de charger chaque page detaillée et recupérer le lien de la page des commentaires

In [ ]:
## Charger la page de chaque machine
def getJSpage(url):
	driver = setupDriver(url)
	html = driver.page_source
	driver.close()
	return BeautifulSoup(html, features="lxml")

Une fois la page des commentaires ouverte, il incombe de déterminer la classe qui sera utilisée pour l'extraction. l'inspection de la page revèle qu'il s'agit d'un objet "div" de classe "bv-content-summary-body-text". 

Dans cette classe, sont présents les commentaires des clients ainsi que ceux de Dell. Nous allons par la suite crire une fonction qui suprime les commentaires de Dell.

Rappelonsque les commentaires de certaines machines peuvent se retrouver sur plus d'une page et doivent être pris en compte. La fonction "getNextPage" dans le code suivant permet donc de recuperer l'url de la page suivante dans la page actuelle afin de pouvoir s'y rendre une fois le scraping sur la page actuelle terminé.

La fonction de recupération des commentaires d'une machine est conçue de façon à fonctionner de façon totalement autonome et adaptative en fonction du nombre de pages de commentaires disponibles pour cette machine. En effet, la fonction recupère les commentaires sur une page, vérifie s'il y a une page suivante et si oui passe à celle-ci et recupère les commentaires aussi. Elle s'arrête quand toutes les pages de commentaires ont t visit et scrapé. La structure du site de Dell est assez bizarre mais nous avons réussi à pouvoir implementer cette fonction.

In [ ]:
##Recuperer l'ensemble des commentaires

# Fonction qui recupère les commentaires d'une page
def getPageComments(soupage):
    cpage = soupage.findAll('div', {'class': 'bv-content-summary-body-text'})
    pagecom = [x.text.strip() for x in cpage]
    return pagecom

# Fonction qui recupère l'URL de la prochaine page de commentaires
def getNextPage(soupage):
    nextPage = soupage.findAll('a', {'class': 'bv-content-btn bv-content-btn-pages bv-content-btn-pages-last bv-focusable bv-content-btn-pages-active'})
    return nextPage

# Fonction de recupration des commentaires pour une machine
def getcomments(NCom,Machinelink):
    PageM = getJSpage(Machinelink)
    Pagecom = []
    Pagecom = Pagecom + getPageComments(PageM)
    nextPage = getNextPage(PageM)
    
    # Vérification de l'existence d'une page suivnate
    while(len(nextPage) != 0):
        PageM2 = getJSpage(nextPage[0]["href"])
        Pagecom = Pagecom + getPageComments(PageM2)
        nextPage = getNextPage(PageM2)
    
    # Création des indices (numéro de commande unique pour chaque machine) et de la fameuse base de données
    MIndex = [NCom]*len(Pagecom)
    comments = pd.DataFrame()
    comments["Comments"] = Pagecom
    comments.index = MIndex
    return comments

Une fois les commentaire récupérés, il faut supprimier les commentaires de Dell. Le souci est que tous les commentaires (clients et Dell) sont dans les mêmes balises et classes. Il est clair qu'utiliser les balises ou les classes pour les séparer allait être très compliqué. Nous avons donc analysé les commentaires de Dell et nous avons remarqu qu'ils avaient tous une signature éléctronique contenant "@Dell" ou "@ Dell".

Ceci nous a données l'idée de scraper tous les commentaires et ensuite de supprimer tous ceux contenant @Dell ou @ Dell

In [ ]:
# Suppression des commentaires de DELL
def DelDellComments(Comments):
    CleanComments = Comments[[(('@Dell' not in str(X))&('@ Dell' not in str(X))) for X in Comments["Comments"]]]
    return CleanComments

Avec toutes ces fonctions, nous pouvons à prsent écrire la fonction de récupérations de tous les commentaires des machines disponibles sur le site de Dell. Cette fonction prend en entrée un Data Frame conténant les codes de commande (identificateurs unique de chaque machine) et les url de toutes les machines et renvoie un Data Frame de commenatires

In [ ]:
# Création de la fonction de recupération de tous les commentaires
def getAllComments(Machines):
    AllComments = pd.DataFrame()
    for i in range(len(Machines)):
        Com = getcomments(Machines.iloc[i,0],Machines.iloc[i,1])
        AllComments = pd.concat([AllComments,Com],axis=0)
    AllComments = DelDellComments(AllComments)
    return AllComments

Le bloc de code suivant permet de récupérer les url de toutes les machines à l'aide d'un scraping classique sur le site de de Dell. Ceci permettra de pouvoir construire le Data Frame néccessaire à la récupération des commentaires comme décrit plus haut. 

In [ ]:
# Catégories de machines et nombre de pages sur le site de Dell

list_ref_categ = [["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/inspiron-laptops",6] , ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/xps-laptops",2], ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/g-series",2] , ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/alienware-laptops",3 ]]

In [ ]:
# Fonction de récupérateion des codes des dmachines et les url de toutes les machines

def url (url):
  el=[]
  codess=[]
  for i  in range (1,int(url[1])+1) :
    urlel= url[0] + "?page=" + str(i) 
    req = request.Request(urlel,headers={'User-Agent': 'Mozilla/5.0'})    
    html = request.urlopen(req).read()
    pagefille= bs4.BeautifulSoup(html,"lxml")
    elementss = pagefille.find_all('div', class_ = "ps-image ps-product-image")  # elementss est une liste dont chaque élément contient une balise de type "a" contenant l'url de la machine considérée
    code = pagefille.find_all('div', class_ = "ps-compare") # code est une liste dont chaque élément contient une balise de type "label" contenant le code de commande d'une machine
    elementss_ref= [element.a['href'] for element in elementss]
    el = el + elementss_ref
    codes=[cod.label['oc'] for cod in code ]
    codess = codess + codes
  return el, codess

In [ ]:
# Récupération proprement dite des codes machines et url

elements=[]
codes=[]

for i in range(len(list_ref_categ)):
  el,codess = url(list_ref_categ[i])
  elements = elements + el
  codes = codes + codess

useful_urls = ["https:" + str(raw_url) + "#ratings_section" for raw_url in elements]

In [ ]:
# Création du Data Frame contenant toutes les machines

All_machines = pd.DataFrame({'code_commande': codes, 'url_ordi':useful_urls}).drop_duplicates()

A ce stade, nous pouvons enfin récupérer les commentaires de toutes les machines sur le site de Dell France...

In [ ]:
# Webscraping des commentaires hahaha
# NB: La fonction utilisée ici supprime aussi les commentaire de Dell et garde juste les commentaires clients

All_comments = getAllComments(All_machines)

In [ ]:
All_comments

Pour la suite des analyse, nous enrégistrons ces commentaires dans un fichier Excel car le scraping prend environ 1h30 et nous ne pourrons pas exécuter le code à chaque fois.

In [ ]:
All_comments.to_excel("C:/Users/nkame/Desktop/ND/COURS ET TD/PYTHON/PROJET/Commentaires.xlsx", header = True)

## SCRAPING DES CARACTERISTIQUES SUR LES LAPTOPS DELL

Il y' a sur le site de DELL France, 4 catégories d'ordinateyrs particuliers. pour wedscrapper leur différentes caractéristiques, il sera question pour nous, pour chaque catégorie, d'accéder au lien de chaque machine, à fin de récupérer les caractéristiques.

Les pages détaillées des machines sont dynamiques et chargées avec Java Script. Les méthodes de scraping basic vu en cours ne suffiront donc pas dans ce cas. Nous utilisons donc dans la suite le package Selenium adapt au scraping des pages dynamique.

Le pilote Web utilisé sera Chrome tout comme dans la partie précédente. Il ne sera donc plus nécéssaire d'importer les bibliothèque. 

Les caractéristiques des ordinateurs dépendent de la catégorie dans laquelle elle se trouve. Donc les fonctions écrites ici dépendront des catégories.

Pour commencer, il nous faut les url des machiines de chaque catégorie.

In [ ]:
list_ref_categ = [ ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/inspiron-laptops",6,'/html/body/main/section[2]/div[1]/div[1]/div[2]/div[2]/span[1]/span','/html/body/main/section[2]/div[2]/div/ul/li'] , ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/xps-laptops",2,'/html/body/main/section[2]/div[1]/div[1]/div[2]/div[2]/span[1]/span','/html/body/main/section[2]/div[2]/div/ul/li'], ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/g-series",2,'//*[@id="cf-strike-through-price"]/div[2]/div','//*[@id="m_146"]/div[2]'] , ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/alienware-laptops",3 ]]

Chaque élément de cette liste est une liste, dont le premier élément est le lien d'une catégorie d'ordinateurs, le second est le nombre de pages web sur lequel s'étend cette catégorie. le troisième et le quatrième élément  sont des "x-path", qui nous aiderons dans le scrapping. 

 

1.    **Fonction de récupération des url  ainsi que des numéros de commandes correspondants des ordinateurs présents sur une page web**




In [ ]:
def url (urlel):   
  driver=webdriver.Chrome('chromedriver', options=options)
  driver.get(urlel) 
  html = driver.page_source
  pagefille= bs4.BeautifulSoup(html,"lxml")
  elementss = pagefille.find_all('div', class_ = "ps-image ps-product-image")  # elementss est une liste dont chaque élément contient une balise de type "a" contenant l'url de la machine considérée
  code = pagefille.find_all('div', class_ = "ps-compare") # code est une liste dont chaque élément contient une balise de type "label" contenant le code de commande d'une machine
  elementss_ref= [[element.a['href']] for element in elementss]   # liste contenant les url des différentes machines de la page web
  for i in range (len(code)):
    elementss_ref[i].append(code[i].label['oc'])
  return elementss_ref

Cette fonction prend en entrée le lien d'une catégorie et nous renvoie une liste. Chaque élément de cette liste est une liste de longeur 2, tel que le premier élément correspond à l'url d'une machine et le deuxième au numéro de commande de ladite machine.

2.   **Fonction de récupération des caractéristiques d'une machines selon sa catégorie.**

Pour le faire,nous ferrons trois fonctions différentes. Une pour les ordinateurs des catégories inspiron et xps, une pour celles de la catégorie g_series et une autre pour celles de la catégorie aliewner.

In [ ]:
# récupération des carctéristiques d'un ordinateir de type inspiron ou xps
def caract_insp_xps (ref,xpath_prix, xpath):
  url_pagefilleel= "http:"+ ref[0] + "#tech-specs-anchor"
  driver=webdriver.Chrome('chromedriver', options=options)
  driver.get(url_pagefilleel)    
  dict_caractel = dict()   # initialisation du dictionnaire des caractéristiques
  caract1= driver.find_elements_by_xpath(xpath_prix) # prix original
  caract1el= caract1[0].text
  dict_caractel['prix']= caract1el# ajouter le prix original dans le dictionnaire des caractéristiques
  xpath_caract= xpath+'/p'
  xpath_name= xpath + '/div'
  nameel= driver.find_elements_by_xpath(xpath_name)
  caractel= driver.find_elements_by_xpath(xpath_caract)
  for i in range(len(caractel)):
    caracte= caractel[i].text
    name= nameel[i].text
    dict_caractel[name]= caracte
  return ( pd.DataFrame.from_dict(dict_caractel,orient='index',columns=[str(ref[1])]))

Cette fonction, pren d en entrée l'url de l'ordinateur ainsi que le "x-path" du prix et des autres caractéristiques. Elle renvoie un data frame dont constitué du numéro de commande de chaque ordinateur ainsi que de ses différentes caractéristiques.  

Nous écrivons le même type de fonction pour les deux autres catégories.

In [ ]:
# récupération des carctéristiques d'un ordinateir de type g-series
def caract_g_series (ref):
  url_pagefilleel= "http:"+ ref[0] + "#tech-specs-anchor"
  driver=webdriver.Chrome('chromedriver', options=options)
  driver.get(url_pagefilleel) 
  html = driver.page_source
  driver.close()
  pagefilleel= bs4.BeautifulSoup(html, features="lxml")
  dict_caractel = dict()  # initialisation du dictionnaire des caractéristiques
  prix= pagefilleel.find('div', class_ = "cf-dell-price").text.strip() #prix
  dict_caractel['prix']= prix # ajout du prix dans le dictionnaire
  name_caract= pagefilleel.find_all('h2', class_ = "ux-module-title")
  processeur= pagefilleel.find('div', class_ = "ux-readonly-title").text.strip()# processeur
  dict_caractel[name_caract[0].text.strip()]= processeur
  sys= pagefilleel.find('div', class_ = "ux-cell-title").text.strip() # système d'exploitation
  dict_caractel[name_caract[1].text.strip()]= sys
  caract= pagefilleel.find_all('div', class_ = "ux-readonly-title ")
  for i in  range(1,6):
    dict_caractel[name_caract[i+1].text.strip()]= caract[i].text.strip()  
  dict_caractel[name_caract[6].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[3].text.strip()
  for i in  range(6, len(caract)):
    dict_caractel[name_caract[i+2].text.strip()]= caract[i].text.strip()  
  dict_caractel[name_caract[len(caract)+1].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[5].text.strip()
  dict_caractel[name_caract[len(caract)+2].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[6].text.strip()
  dict_caractel[name_caract[len(caract)+3].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[7].text.strip()
  dict_caractel[name_caract[len(caract)+4].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[8].text.strip()
  dict_caractel[name_caract[len(caract)+5].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[9].text.strip()
  dict_caractel[name_caract[len(caract)+6].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[10].text.strip()
  return( pd.DataFrame.from_dict(dict_caractel,orient='index',columns=[str(ref[1])]) )


In [ ]:
# récupération des carctéristiques d'un ordinateir de type aliewner
def caract_aliewener (ref):
  url_pagefilleel= "http:"+ ref[0] + "#tech-specs-anchor"
  driver=webdriver.Chrome('chromedriver', options=options)
  driver.get(url_pagefilleel) 
  html = driver.page_source
  driver.close()
  pagefilleel= bs4.BeautifulSoup(html, features="lxml")
  dict_caractel = dict()  # initialisation du dictionnaire des caractéristiques
  prix= pagefilleel.find('div', class_ = "cf-dell-price").text.strip() #prix
  dict_caractel['prix']= prix # ajout du prix dans le dictionnaire
  name_caract= pagefilleel.find_all('h2', class_ = "ux-module-title")
  caract= pagefilleel.find_all('div', class_ = "ux-readonly-title ")
  for i in  range(len(caract)):
    dict_caractel[name_caract[i].text.strip()]= caract[i].text.strip() 
  #for i in range(3,len(caract)):
    #dict_caractel[name_caract[i].text.strip()]= caract[i].text.strip()
  dict_caractel[name_caract[len(caract)+1].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[1].text.strip()
  dict_caractel[name_caract[len(caract)+2].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[2].text.strip()
  dict_caractel[name_caract[len(caract)+3].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[3].text.strip()
  dict_caractel[name_caract[len(caract)+4].text.strip()]= pagefilleel.find_all('div', class_ = "ux-cell-title")[4].text.strip()
  return( pd.DataFrame.from_dict(dict_caractel,orient='index',columns=[str(ref[1])]) )

Ces deux fonctions prennent en paramètre uniquement l'url d'un ordinateur. La méthode avec le "x-path" ne fonctionne pas ici , car, chaque caractéristique a son "x-path". Il est donc difficile de généraliser.

Il est question dans la suite de récupérer les caractéristiques de tous les ordinateurs présents sur une page web, selon la catégorie.

3.    **Fonction de récupération des caractéristiques des machines sur une page web selon sa catégorie.**

In [ ]:
# récupération des carctéristiques  des ordinateurs de type inspiron ou xps d'une page web donnée.
def base_inter_insp_xps ( url_page,xpath_prix,xpath_caract):
  elementss_ref= url(url_page)   # renvoie la liste l'url des machines sur url_page et les codes de commandes associés  
  data1= pd.DataFrame()      # initialisation du dataframe qui comportera  les machine d'une pageweb ainsi que leurs caractéristiques
  for i in range (len(elementss_ref)):
    data_inter= caract_insp_xps(elementss_ref[i],xpath_prix,xpath_caract)  # dataframe d'un ordi avec ses caractéristiques
    data1= pd.concat([data1, data_inter],axis=1,sort=True)     
  return data1

Cette fonction prend en  entrée l'url de la page web et renvoie un data frame, constitué des numéros de commandes et descaractéristiques des ordinateurs présentes sur la page web correspondante.

Nous faisons le même type de fonction pour les deux autres catégories.

In [ ]:
# récupération des carctéristiques  des ordinateurs de type g-series d'une page web donnée.
def base_inter_gseries ( url_page):
  elementss_ref= url(url_page)   # renvoie la liste l'url des machines sur url_page et les codes de commandes associés
  data1= pd.DataFrame()  # initialisation du dataframe qui comportera  les machine d'une pageweb ainsi que leurs caractéristiques
  for i in range (len(elementss_ref)):
    data_inter= caract_g_series(elementss_ref[i])  # dataframe d'un ordi avec ses caractéristiques
    data1= pd.concat([data1, data_inter],axis=1,sort=True) 
  return data1

In [ ]:
# récupération des carctéristiques  des ordinateurs de type aliewner  d'une page web donnée.
def base_inter_aliewner ( url_page):
  elementss_ref= url(url_page)   # renvoie la liste l'url des machines sur url_page et les codes de commandes associés
  data1= pd.DataFrame()   # initialisation du dataframe qui comportera  les machine d'une pageweb ainsi que leurs caractéristiques
  for i in range (len(elementss_ref)):
    data_inter= caract_aliewener(elementss_ref[i])  # liste des caractéristiques de chaque ordimateur
    data1= pd.concat([data1, data_inter],axis=1,sort=True)   
  return data1

Après avoir contitué nos bases intermédiaires( pour une seule page web), il est question maintenant les concatener pour avoir notre base finale selon les catégories. Pour le faire, nous avons besoin d'une fonction qui pour un type de machine, va sur toutes les pages qui lui sont associées.

In [ ]:
# récupération des caractéristiques des ordinateurs de type inspiron ou xps
def base_finale_insp_xps (url_page):
  base= pd.DataFrame()  # initialisation du dataframe
  for i  in range (1,int(url_page[1])+1) :
    url_pagefille1= url_page[0] + "?page=" + str(i)  # ième page web de la catégorie dont l'url est "url_page"
    inter= base_inter_insp_xps(url_pagefille1,url_page[2],url_page[3])   
    base = pd.concat([base,inter], axis=1)
  return base

In [ ]:
# récupération des caractéristiques des ordinateurs de type g_series
def base_finale_gseries (url_page):
  base= pd.DataFrame()  # initialisation du dataframe
  for i  in range (1,int(url_page[1])+1) :
    url_pagefille1= url_page[0] + "?page=" + str(i)  # ième page web de la catégorie dont l'url est "url_page"
    inter= base_inter_gseries(url_pagefille1)   
    base = pd.concat([base,inter], axis=1)
  return base

In [ ]:
# récupération des caractéristiques des ordinateurs de type aliewner
def base_finale_aliewnere (url_page):
  base= pd.DataFrame()  # initialisation du dataframe
  for i  in range (1,int(url_page[1])+1) :
    url_pagefille1= url_page[0] + "?page=" + str(i)  # ième page web de la catégorie dont l'url est "url_page"
    inter= base_inter_aliewnere(url_pagefille1)   # 
    base = pd.concat([base,inter], axis=1)
  return base

Les fonction ci-dessus , prennnent en paramètre, une liste dont le premier élément est l'url de la première page d'une catégorie la deuxième est le nombre de pages sur lesquelle elle tient et renvoie la base constituée des numéros de commande de toutes les machines d'une catégorie ainsi que leurs caractéristiques.

 4. **Application de nos fonction et récupération des bases**

In [ ]:
# Constitution de la base des caractéristiques pour les catégories inspirons et xps
data_inpiron_xps = pd.dataFrame()
for i  in range (2) :
    data_inter = base_finale_insp_xps(list_ref_categ[i])
    data_inpiron_xps = data_inpiron_xps.append(data_inter)
    


In [ ]:
# Constitution de la base des caractéristiques pour les catégories g_series
data_g_series= base_finale_gseries(list_ref_categ[2])

In [ ]:
# Constitution de la base des caractéristiques pour les catégories aliewner
data_aliewner = base_finale_aliewnere(list_ref_categ[3])

Pour la suite des analyse, nous enrégistrons ces caractéristiques dans des fichiers csv car le scraping prend environ 1h30 et nous ne pourrons pas exécuter le code à chaque fois.

In [ ]:
data_inpiron_xps.to_csv('data_inpiron_xps.csv')
data_g_series.to_csv('data_g_series.csv')
data_aliewner.to_csv('data_aliewner.csv')